In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')

In [7]:
customers['SignupYear'] = pd.to_datetime(customers['SignupDate']).dt.year
customers['Age'] = 2025 - customers['SignupYear']  # Replace 2025 with the current year


In [11]:
customers['Income'] = customers['Region'].map({'Region1': 50000, 'Region2': 60000}).fillna(55000)


In [8]:
customer_features = customers[['CustomerID', 'Region', 'Age']]

In [12]:
# Simulated Example for Transaction Aggregates
customer_features['TotalSpent'] = customers['Income'] * 0.1  # Placeholder
customer_features['TransactionCount'] = customers['Age'] % 5 + 1  # Placeholder

C:\Users\amanp\AppData\Local\Temp/ipykernel_4288/3897186950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_features['TotalSpent'] = customers['Income'] * 0.1  # Placeholder
C:\Users\amanp\AppData\Local\Temp/ipykernel_4288/3897186950.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_features['TransactionCount'] = customers['Age'] % 5 + 1  # Placeholder


In [14]:
customer_features_encoded = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)


In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
customer_features['Region'] = label_encoder.fit_transform(customer_features['Region'])


C:\Users\amanp\AppData\Local\Temp/ipykernel_4288/3444557086.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_features['Region'] = label_encoder.fit_transform(customer_features['Region'])


In [16]:
# Normalize the features for similarity calculation
scaler = StandardScaler()
customer_features_scaled = pd.DataFrame(
    scaler.fit_transform(customer_features.iloc[:, 1:]),  # Exclude CustomerID from scaling
    columns=customer_features.columns[1:],
    index=customer_features['CustomerID']
)

In [17]:
# Compute Similarity Matrix
similarity_matrix = cosine_similarity(customer_features_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

In [18]:
similarity_matrix = cosine_similarity(customer_features_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

In [19]:
# Finding top 3 lookalikes for the first 20 customers (CustomerID: C0001 - C0020)
top_20_customers = customers.loc[customers['CustomerID'].str.startswith('C00'), 'CustomerID'][:20]
lookalike_results = {}



In [20]:
for cust_id in top_20_customers:
    # Getting the similarity scores for the customer
    similar_customers = similarity_df[cust_id].sort_values(ascending=False).iloc[1:4]  # Exclude the customer itself
    lookalike_results[cust_id] = list(zip(similar_customers.index, similar_customers.values))



In [21]:
# Preparing Lookalike.csv
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_results.keys(),
    'Lookalikes': [str(v) for v in lookalike_results.values()]
})